### Q1: What is the average number of sets per year?


To answer this query, a temporary view `sets_per_year` is defined. The view contains a record for each `year` with existing `sets`, and the number of sets, `num_sets_per_year`, produced in that year. The average number of sets per year is computed using this temporary view.

In [62]:
WITH sets_per_year AS
(SELECT year, 
       COUNT(*) AS num_sets_per_year
FROM sets
GROUP BY year)
SELECT *
FROM sets_per_year          

,year,num_sets_per_year
0,1964,11
1,1969,69
2,2008,349
3,1989,115
4,1991,107
...,...,...
61,1955,28
62,2012,615
63,1954,14
64,1959,4


In [63]:
WITH sets_per_year AS
(SELECT year, 
       COUNT(*) AS num_sets_per_year
FROM sets
GROUP BY year)
SELECT ROUND(AVG(num_sets_per_year), 2) AS avg_num_sets_per_year
FROM sets_per_year          


,avg_num_sets_per_year
0,176.86


### Q2: What is the average number of parts per year?

The average number of parst per year is `1056.5`.

In [64]:
WITH parts_per_year AS
(SELECT s.year, COUNT(DISTINCT p.part_num) as parts_per_year
FROM parts p JOIN inventory_parts ip ON p.part_num = ip.part_num 
JOIN inventories iv ON ip.inventory_id = iv.id
JOIN sets s ON s.set_num = iv.set_num
GROUP BY s.year
)    
SELECT ROUND(AVG(parts_per_year), 2) AS avg_parts_per_year
FROM parts_per_year          


,avg_parts_per_year
0,1056.5


In [65]:
WITH parts_per_year AS
(SELECT s.year, COUNT(DISTINCT p.part_num) AS parts_per_year
FROM parts p JOIN inventory_parts ip ON p.part_num = ip.part_num 
JOIN inventories iv ON ip.inventory_id = iv.id
JOIN sets s ON s.set_num = iv.set_num
GROUP BY s.year
)    
SELECT year, AVG(parts_per_year) AS avg_parts_per_year
FROM parts_per_year
GROUP BY year

,year,avg_parts_per_year
0,1950,6.0
1,1953,6.0
2,1954,27.0
3,1955,86.0
4,1956,47.0
...,...,...
61,2013,3786.0
62,2014,3726.0
63,2015,4166.0
64,2016,4297.0


### Q3: What is the average number of parts per year? Create a visual.

From the line graph below, it can be noticed that the average number of parts per year has increased consistently year after year. A few noticeable drops happened in 2005, 2007 and 2017, with the one in 2017 being the largest drop. This last drop, from `4297` to `3013` is significant, nearing a `25%` drop.

,year,avg_parts_per_year
0,1950,6.0
1,1953,6.0
2,1954,27.0
3,1955,86.0
4,1956,47.0
...,...,...
61,2013,3786.0
62,2014,3726.0
63,2015,4166.0
64,2016,4297.0


### Q4: What are the 5 most popular colors used in Lego parts?

The first step to answering this question is providing some clarity to the meaning of _**popular color**_. Based on the available information, a color is popular either  
- when there are many different parts in that color existing in the inventory **(a)**, or
- when the quantity of parts is large in that particular color **(b)**.

It is unclear to me which one of these interpretations of the popular color is preferred, so a solution is presented for each of them.

Note that even though the `colors` table has the attribute `id` as the unique identifier, the `name` attribute is also unique among the records of the `colors` table. To verify this, the following queries were executed:
- `SELECT COUNT(DISTINCT name) FROM colors`, and
- `SELECT COUNT(*) FROM colors`
Both queries return the same result, `135`.

The two solutions presented below return the results for the top 5 favourite colors. A visual inspection of the two charts reveals that the ratios are very similar between them.


In [67]:
SELECT c.name, COUNT(ip.part_num) AS num_parts
FROM colors c JOIN inventory_parts ip ON c.id = ip.color_id
GROUP BY c.name
ORDER BY num_parts DESC
LIMIT 5


,name,num_parts
0,Black,115085
1,White,66536
2,Light Bluish Gray,55302
3,Red,50213
4,Dark Bluish Gray,43907


NameError: name 'parts_colors_a' is not defined

In [69]:
SELECT c.name, SUM(ip.quantity) AS num_parts
FROM colors c JOIN inventory_parts ip ON c.id = ip.color_id
GROUP BY c.name
ORDER BY num_parts DESC
LIMIT 5

,name,num_parts
0,Black,396416
1,White,221553
2,Light Bluish Gray,201858
3,Red,169983
4,Dark Bluish Gray,148584


In [70]:
# This is a chart, switch to the DataCamp editor to view and configure it.

,name,num_parts
0,Black,396416
1,White,221553
2,Light Bluish Gray,201858
3,Red,169983
4,Dark Bluish Gray,148584


### Q5: What proportion of the lego parts are transparent?

A _**transparent**_ part is one that has a transparent color, indicated by the `is_trans` attribute of the `colors` table. The same part (`part_num`) can be ordered in various colors, either transparent or not. Note that there are `1455` parts that are not part of the `inventory_parts`, so no conclusion can be drawn with respect to these ones.

To answer this question, either the quantity of the parts or the number of distinct parts can be used as measure. If the quantities of the parts is used, the following formula is applied:

$$TransparentParts = \frac{QuantityOfTransparentParts}{QuantityOfAllParts}.$$

Alternatively, if the number of distinct parts is used, the following formula is applied:

$$TransparentParts = \frac{NumberOfTransparentParts}{NumberOfAllParts}.$$

Two solutions are presented, one for each of the formulas above. Interestingly, although each solution returns a different result, they are very close in range. Since a part can be ordered in various colors, the difference in the two numbers can be explained as an overcounting of the parts.

- When the first formula is used, `4.94%` of the Lego parts are transparent.
- When the second formula is used, `6.29%` of the Lego parts are transparent.

When the second formula is used, the parts can be counted in different ways: considering their color, or disregarding the color. That is, part `(A, red)` is different than part `(A, blue)`. When the counting is done in this way, a different final numbers are obtained, `6.11%`. Note that  `6.11%` and `6.29%` are very close in range, indicating that overcounting is less prevalent in this case.

In [71]:
WITH quantity_all AS
(SELECT SUM(quantity) AS total
 FROM inventory_parts
),
quantity_transparent_parts AS
(SELECT SUM(ip.quantity) AS trans_parts
 FROM colors c JOIN inventory_parts ip ON c.is_trans = True AND c.id = ip.color_id 
)
SELECT ROUND(trans_parts * 100.0/total, 2)
FROM quantity_all, quantity_transparent_parts

,round
0,4.94


In [76]:
WITH all_parts AS
(SELECT COUNT(DISTINCT part_num) AS total
 FROM inventory_parts
),
transparent_parts AS
(SELECT COUNT(DISTINCT part_num) AS trans_parts
 FROM colors c JOIN inventory_parts ip ON c.is_trans = True AND c.id = ip.color_id 
)
SELECT ROUND(trans_parts * 100.0/total ,2)
FROM all_parts, transparent_parts

,round
0,6.29


In [73]:
WITH all_parts AS
(SELECT COUNT(*)  AS total 
FROM
(SELECT part_num, color_id
 FROM inventory_parts
 GROUP BY part_num, color_id
) T
),
transparent_parts AS
(SELECT COUNT(*)  AS total_transparent
 FROM
(SELECT part_num, color_id
 FROM colors c JOIN inventory_parts ip ON c.is_trans = True AND c.id = ip.color_id 
 GROUP BY part_num, color_id
) T
)
SELECT ROUND(total_transparent * 100.0/total ,2)
FROM all_parts, transparent_parts


,round
0,6.11


### Q6: What are the rarest Lego bricks?

An analysis of the `part_categories` table reveals that there are 8 different `Bricks` categories. 

There are different ways in which we can measure a brick's rarity. One way is to check each of the 8 categories of bricks and choose the one with the smallest number of parts in that category. `Technic Bricks` is the category with the smallest number of parts, `26`. 

Another way to define the rarity of bricks is by checking the representation of all categories of bricks in the inventory. There are `838` rare bricks, that appear just once in the inventory.



In [74]:
SELECT pc.name, COUNT(*) AS ctn
FROM part_categories pc JOIN parts p ON pc.name LIKE '%Brick%' AND pc.id = p.part_cat_id
GROUP BY pc.id
ORDER BY ctn

,name,ctn
0,Technic Bricks,26
1,Bricks,93
2,Bricks Special,116
3,Bricks Wedged,198
4,Bricks Round and Cones,219
5,Bricks Curved,275
6,Bricks Sloped,410
7,Bricks Printed,744


In [75]:
WITH min_count AS(
SELECT COUNT(*) AS ctn
FROM inventory_parts
GROUP BY part_num
ORDER BY ctn
LIMIT 1
)
SELECT p.name
FROM part_categories pc JOIN parts p ON pc.name LIKE '%Brick%' AND pc.id = p.part_cat_id
JOIN inventory_parts ip ON p.part_num = ip.part_num
GROUP BY p.name
HAVING COUNT(*) = (SELECT ctn FROM min_count)




,name
0,Brick 1 x 4 with Belle Dress print
1,"BRICK 2X2, NO. 104 Horseshoe Number 1"
2,"ROUND BRICK 2X2X2, &quot;NO 1006&quot;"
3,"Slope, Curved 2 x 2 Lip, No Studs with Red Che..."
4,Brick 1 x 2 with Red Sterling S Print (1551)
...,...
833,Brick 2 x 4 with Angled Red Stripes Between Tw...
834,Hemisphere 11 x 11 - 4 Studs on Top with Endor...
835,Slope Brick 33 3 x 6 with SW ARC-170 Left Print
836,Brick 1 x 6 with Black 'CATERHAM' on Yellow St...


## Summary

1. The first thing I noticed when analyzing the data is the inefficient schema design. For instance, there is no simple way to find out what are the parts belonging to a Lego set, as there is no direct relationship between the two tables. To answer this simple question, one needs to look into inventories. Maybe there are other considerations, outside the scope of this exercise, that resulted into this particular schema design.
2. Another consequence of the design is the ambiguity related to measures. For instance, when we are checking for certain properties of parts, do we count the quantities of that part in inventories, or do we count how many times that part was ordered?
3. This was a fun exercise, thank you!